In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as utils_data

In [2]:
# ALL GLOBAL VARIABLES

GLOBAL_DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
GLOBAL_SEED = 42
PADDING_CONST = 0

In [3]:
!nvidia-smi

Tue Dec  8 22:14:29 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    27W / 250W |     10MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
torch.cuda.empty_cache()

In [5]:
def compute_integral_mc(intensity_network, hidden_states, src_padding_mask, time, events, alpha=-0.1, n_samples=100):
    """
    Compute integral using Monte Carlo integration.
    """
    # time differences (t_{j} - t{j-1})
    dt = (time[:, 1:] - time[:, :-1]) * (~src_padding_mask[:, 1:])

    # sample t from uniform distribution: 
    # since t \in (t_{j-1}, t_j) which would lead to (t - t_{j-1}) / t_{j-1} \in [0, (t_j - t_{j-1}) / t_{j-1}),
    # we can reformulate this as (t_j - t_{j-1}) / t_{j-1} * u, where u \in [0, 1)
    current_influence = alpha * (dt.unsqueeze(2) / (time[:, :-1] + 1).unsqueeze(2)) * torch.rand([*dt.size(), n_samples], device=hidden_states.device)

    # compute sum( lambda(u_i) ) / N
    mc_intensity = intensity_network(hidden_states, events, current_inf=current_influence, mc_trick=True)

    return mc_intensity * dt

def compute_integral_li(lam, time, src_padding_mask):
    dt = (time[:, 1:] - time[:, :-1]) * (~src_padding_mask[:, 1:])
    dlam = (lam[:, 1:] + lam[:, :-1]) * (~src_padding_mask[:, 1:])

    integral = dt * dlam
    return 0.5 * integral

In [6]:
class IntensityNetwork(nn.Module):

    def __init__(self, hidden_size, n_event_types, device):
        """
        Inputs:
            hidden_size (int) - size of the hidden dimension (d_model),
            n_event_types (int) - number of possible event types in the data
        """
        super(IntensityNetwork, self).__init__()

        self.n_events = n_event_types
        self.device = device

        # accounts for "history" and "base" (through bias) terms in eq.(6) of the paper
        self.linear = nn.Linear(hidden_size, n_event_types)
        self.softplus = nn.Softplus(threshold=10)

    def generate_type_mask(self, events):
        bs, ls = events.size()

        type_mask = torch.zeros(bs, ls, self.n_events, device=self.device)
        for k in range(self.n_events):
            type_mask[:, :, k] = (events == k + 1).bool().to(self.device)
        return type_mask
    
    def forward(self, hidden_states, events, current_inf=None, mc_trick=False):
        intensity_terms = self.linear(hidden_states)
        type_mask = self.generate_type_mask(events)

        if mc_trick:
            # this is a trick for Monte-Carlo integration, which allows to vectorize
            # computation of (num_samples) intensity functions instead of making a loop

            assert current_inf is not None, "current influence cannot be None when mc_trick is True"

            intensity_terms = (intensity_terms[:, :-1, :] * type_mask[:, :-1, :]).sum(dim=2, keepdim=True)
            continious_intensity = self.softplus( intensity_terms + current_inf )
            conditional_lambda = continious_intensity.mean(dim=2)
        else:
            if current_inf is not None:
                intensity_terms += current_inf
            continious_intensity = self.softplus(intensity_terms)

            # (continious_intensity * type_mask) gets type-specific instensity function (eq. (6))
            # after summation along the 2nd dimension, conditional intensity function is obtained
            conditional_lambda = (continious_intensity * type_mask).sum(dim=2)
        
        return conditional_lambda

In [7]:
class HawkesTransformer(nn.Module):

    def __init__(self, n_event_types, device, d_model=512, n_heads=8, dim_feedforward=2048, n_layers=6, dropout=0.1, activation='relu'):
        """
        Input parameters:
            n_event_types (int) - number of event types in the data,
            d_model (int) - size of model's latent dimension,
            n_heads (int) - number of heads in the Multihead Attention module,
            dim_feedforward (int) - size of the feedforward network dimension,
            n_layers (int) - number of Transformer encoder layers,
            dropout (float) - dropout rate,
            activation (string) - activation function for the feedforward network (relu or gelu)
        """
        super(HawkesTransformer, self).__init__()

        self.n_events = n_event_types
        self.d_model = d_model
        self.device = device

        # initialize div term for temporal encoding
        self.init_temporal_encoding()

        # event type embedding
        self.event_embedding = nn.Embedding(n_event_types + 1, d_model, padding_idx=PADDING_CONST)

        # transformer encoder layers
        encoder_layer = nn.TransformerEncoderLayer(d_model, n_heads, dim_feedforward, dropout, activation)
        self.transformer_layers = nn.TransformerEncoder(encoder_layer, n_layers)

        # linear transformation of hidden states ("history" and "base" terms in eq.(6) of the THP paper) to
        # type specific intensity
        self.intensity_layer = IntensityNetwork(d_model, n_event_types, self.device)

        # output prediction layers
        self.time_predictor  = nn.Linear(d_model, 1, bias=False)
        self.event_predictor = nn.Linear(d_model, n_event_types, bias=False)

        # small constant
        self.eps = torch.tensor([1e-9], device=self.device)

    def generate_subsequent_mask(self, seq):
        """
        Function to generate masking for the subsequent information in the sequences (masked self-attention).
        Input:
            seq (B, S) - batch of sequences.
        """
        bs, ls = seq.size()
        subsequent_mask = torch.triu( torch.ones(ls, ls, device=self.device, dtype=torch.bool), diagonal=1 )
        
        return subsequent_mask
    
    def generate_key_padding_mask(self, seq):
        """
        Masking the padded part of the sequence.
        Input:
            seq (B, S) - batch of sequences.
        """
        padding_mask = seq.eq(PADDING_CONST)

        return padding_mask

    def init_temporal_encoding(self):
        """
        Initializing the internal temporal encoding tensors.
        """
        encoding_constant = torch.tensor(10000.0)

        # for better numerical stability
        self.te_div_term = torch.exp(2.0 * (torch.arange(0, self.d_model) // 2) * -torch.log(encoding_constant) / self.d_model).to(self.device)
  
    def temporal_encoding(self, t, non_padded_mask):
        """
        Function to perform the temporal encoding on input timestamps.
        Input:
            t (B, S) - batch of timestamp sequences,
            non_padded_mask (B, S) - binary mask indicating whether element is a padding (True) or not (False)
        Output:
            x (B, S, d_model) - raw model output,
            lam (B, S, F) - intensity function,
            time_pred (B, S) - timestamp prediction for the next event,
            event_pred (B, S, n_event_types) - probabilities of event types
        """
        temporal_enc = t.unsqueeze(-1) * self.te_div_term

        temporal_enc[:, :, 0::2] = torch.sin(temporal_enc[:, :, 0::2])
        temporal_enc[:, :, 1::2] = torch.cos(temporal_enc[:, :, 1::2])

        return temporal_enc * non_padded_mask.unsqueeze(-1)
    
    def log_likelihood(self, hidden_states, cond_lam, time, events, alpha, integral='mc'):
        """
        Inputs:
            hidden_states (B, S, E) - hidden states of the network,
            cond_lam (B, S, F) - conditional intensity function,
            time (B, S) - ground truth for times,
            events (B, S) - ground truth for event types,
            alpha (int) - scaling constant,
            integral (string) - method of integration: either Monte-Carlo or linear interpolation
        """
        
        src_padding_mask = self.generate_key_padding_mask(events)

        # # compute event log-likelihood
        event_part = cond_lam + self.eps
        event_part.masked_fill_(src_padding_mask, 1.0)
        event_part = event_part.log()
        event_part = event_part.sum(dim=1)

        # # compute non-event log-likelihood
        if integral == 'mc':
            non_event_part = compute_integral_mc(self.intensity_layer, hidden_states, src_padding_mask, time, events, alpha)
        else:
            non_event_part = compute_integral_li(cond_lam, time, src_padding_mask)
        non_event_part = non_event_part.sum(dim=1)

        # # compute total log-likelihood
        log_likelihood = (event_part - non_event_part).sum()

        return log_likelihood

    def time_error(self, time_pred, time):
        """
        Inputs:
            time_pred (B, S) - time predictions,
            time (B, S) - ground truth for times
        """

        time_ground_truth = time[:, 1:] - time[:, :-1]
        time_pred = time_pred[:, :-1]

        time_error = nn.MSELoss(reduction='sum')(time_pred, time_ground_truth)
        return time_error

    def event_error(self, event_pred, events):
        """
        Inputs:
            event_pred (B, S, K) - event probabilities predictions,
            events (B, S) - ground truth for event types,
        """

        event_ground_truth = events[:, 1:] - 1
        event_pred = event_pred[:, :-1, :]

        event_error = nn.CrossEntropyLoss(reduction='sum', ignore_index=-1)(event_pred.transpose(1, 2), event_ground_truth)
        return event_error
    
    def forward(self, time, events):
        """
        Input:
            time (B, S) - input sequence of event timestamps,
            events (B, S) - input sequence of event types
        """

        # generate masks
        src_key_padding_mask = self.generate_key_padding_mask(events)
        src_non_padded_mask = ~src_key_padding_mask
        src_mask = self.generate_subsequent_mask(events)

        # perform encodings
        temp_enc  = self.temporal_encoding(time, src_non_padded_mask)
        event_enc = self.event_embedding(events)

        # make pass through transformer encoder layers
        x = event_enc + temp_enc
        h = self.transformer_layers(x.permute(1, 0, 2), mask=src_mask, src_key_padding_mask=src_key_padding_mask)
        h = h.permute(1, 0, 2)

        # obtain conditional intensity function
        cond_lam = self.intensity_layer(h, events)

        # make predictions
        time_pred  = self.time_predictor(h).squeeze(2) * src_non_padded_mask
        event_pred = self.event_predictor(h) * src_non_padded_mask.unsqueeze(-1)

        return h, cond_lam, time_pred, event_pred

In [8]:
def run_epoch(model, dataloader, device, optimizer=None):
    
    ll_loss_epoch, tp_loss_epoch, ec_loss_epoch = 0., 0., 0.
    event_num_total, event_pred_correct, event_pred_total = 0, 0, 0

    with torch.set_grad_enabled(optimizer is not None):
        for time, events in dataloader:
            time = time.to(device)
            events = events.to(device)

            h, cond_lam, time_pred, event_pred = model(time, events)

            # Log-likelihood (loss for the whole sequence)
            loss_ll = model.log_likelihood(h, cond_lam, time, events, -0.1, 'mc')

            # Time prediction loss
            loss_tp = model.time_error(time_pred, time)

            # Event type classficiation loss
            loss_ec = model.event_error(event_pred, events)

            # Combined loss function
            scale = 0.01
            loss = -loss_ll + loss_ec + loss_tp * scale

            if optimizer is not None:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            # Logging
            ll_loss_epoch += loss_ll.item()
            tp_loss_epoch += loss_tp.item()
            ec_loss_epoch += loss_ec.item()

            batch_num_events = events.ne(PADDING_CONST).sum().item()
            event_num_total += batch_num_events
            event_pred_correct += (event_pred[:, :-1, :].argmax(dim=2) == events[:, 1:] - 1).sum().item()
            event_pred_total += batch_num_events - events.shape[0]
    
    ll_loss_epoch /= event_num_total
    tp_loss_epoch /= event_pred_total
    ec_loss_epoch /= event_pred_total
    accuracy = event_pred_correct / event_pred_total

    return ll_loss_epoch, tp_loss_epoch, ec_loss_epoch, accuracy

In [9]:
import time as t

def train(model, n_epochs, optimizer, train_loader, val_loader, scheduler=None, device=None, verbose=True, freq=None):
    if verbose and freq is None:
        freq = max(n_epochs // 10, 1)
    if device is None:
        device = GLOBAL_DEVICE

    train_loss_ll_history, train_loss_tp_history, train_loss_ec_history, train_accuracy_history = [], [], [], []
    val_loss_ll_history, val_loss_tp_history, val_loss_ec_history, val_accuracy_history = [], [], [], []

    time_start = t.time()
    for epoch in range(1, n_epochs + 1):
        model.train()
        train_loss_ll, train_loss_tp, train_loss_ec, train_accuracy = run_epoch(model, train_loader, device, optimizer)
        train_loss_ll_history.append( train_loss_ll )
        train_loss_tp_history.append( train_loss_tp )
        train_loss_ec_history.append( train_loss_ec )
        train_accuracy_history.append( train_accuracy )

        model.eval()
        val_loss_ll, val_loss_tp, val_loss_ec, val_accuracy = run_epoch(model, val_loader, device)
        val_loss_ll_history.append( val_loss_ll )
        val_loss_tp_history.append( val_loss_tp )
        val_loss_ec_history.append( val_loss_ec )
        val_accuracy_history.append( val_accuracy )

        if scheduler is not None:
            scheduler.step()
        
        time_epoch_end = t.time() - time_start

        if verbose and epoch % freq == 0:
            print("[ Epoch {} ]".format(epoch))
            print("(Training)     log-likelihood: {}, RMSE: {}, CE: {}, accuracy: {}".format( train_loss_ll, np.sqrt(train_loss_tp), train_loss_ec, train_accuracy ))
            print("(Validation)   log-likelihood: {}, RMSE: {}, CE: {}, accuracy: {}".format( val_loss_ll, np.sqrt(val_loss_tp), val_loss_ec, val_accuracy ))
            print("Time elapsed: {:.2f} s".format(time_epoch_end))

    train_history = {
        'log-likelihood' : train_loss_ll_history,
        'time mse' : train_loss_tp_history,
        'cross entropy' : train_loss_ec_history,
        'accuracy' : train_accuracy_history
    }

    val_history = {
        'log-likelihood' : val_loss_ll_history,
        'time mse' : val_loss_tp_history,
        'cross entropy' : val_loss_ec_history,
        'accuracy' : val_accuracy_history
    }
    return train_history, val_history

In [10]:
#!unzip -q fin_data.zip -d data

In [11]:
import pickle

class NHPDataset(utils_data.Dataset):
    ''' 
    Create Dataset for Neural Hawkey Process
    '''

    def __init__(self, file_path):
        self.event_type = []
        self.event_time = []

        with open(file_path, 'rb') as f:

            if 'dev' in file_path:
                seqs = pickle.load(f, encoding='latin1')['dev']
            elif 'train' in file_path:
                seqs = pickle.load(f, encoding='latin1')['train']
            elif 'test' in file_path:
                seqs = pickle.load(f, encoding='latin1')['test']

            for idx, seq in enumerate(seqs):
                self.event_type.append(torch.Tensor([int(event['type_event']) for event in seq]))
                self.event_time.append(torch.Tensor([float(event['time_since_start']) for event in seq]))

    def __len__(self):
        return len(self.event_type)
    
    def __getitem__(self, index):

        event_type = torch.LongTensor(self.event_type[index].long() + 1)
        event_time = torch.Tensor(self.event_time[index])
        seq_len = torch.tensor(len(event_type))
        event_last_time = torch.sum(event_time)

        X = torch.stack((event_time, event_type), dim=1)
        
        return event_time, event_type

In [12]:
train_dataset = NHPDataset('data/train.pkl')
val_dataset = NHPDataset('data/test.pkl')

train_loader = utils_data.DataLoader(train_dataset, batch_size=5, shuffle=True)
val_loader = utils_data.DataLoader(val_dataset, batch_size=5, shuffle=False)

In [13]:
model = HawkesTransformer(2, GLOBAL_DEVICE, 512, 4, 1024, 4, 0.1, 'relu').to(GLOBAL_DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, betas=(0.9, 0.999), eps=1e-05)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 10, gamma=0.5)

train_history, val_history = train(model, 50, optimizer, train_loader, val_loader, scheduler, GLOBAL_DEVICE, freq=1)

[ Epoch 1 ]
(Training)     log-likelihood: -1.268742206069432, RMSE: 20.24977521151604, CE: 0.768273220224491, accuracy: 0.5478601567209163
(Validation)   log-likelihood: -1.3759121199770055, RMSE: 17.456689786167843, CE: 0.6703899379859224, accuracy: 0.6134782608695653
Time elapsed: 10.79 s
[ Epoch 2 ]
(Training)     log-likelihood: -1.2351691668706688, RMSE: 20.2477570646395, CE: 0.6827322709201912, accuracy: 0.5839762909383163
(Validation)   log-likelihood: -1.292960654190855, RMSE: 17.459266752290223, CE: 0.6656653598656401, accuracy: 0.6160869565217392
Time elapsed: 21.54 s
[ Epoch 3 ]
(Training)     log-likelihood: -1.1781854846935573, RMSE: 20.23963673672891, CE: 0.6626432729880366, accuracy: 0.6203301855200589
(Validation)   log-likelihood: -1.2988057959985675, RMSE: 17.46235149840766, CE: 0.6671821277268267, accuracy: 0.6152777777777778
Time elapsed: 32.30 s
[ Epoch 4 ]
(Training)     log-likelihood: -1.1237578227001104, RMSE: 20.233397889223127, CE: 0.6654452964103627, accura

In [14]:
torch.save(model.state_dict(), 'model.pth')